# Model Training and Inference with PROTOplast for the Virtual Cell Challenge
This tutorial walks through the process of building a PyTorch Lightning model and training it using the PROTOplast for the Virtual Cell Challenge (VCC). It also covers loading a trained model from a checkpoint for inference, making predictions, writing the results into an AnnData object, and preparing a submission using the `cell-eval` tool.

In [1]:
import anndata
import glob
import numpy as np
import pandas as pd
import os
import pathlib
import protoplast as pt
import ray
import torch

from anndata.experimental import AnnCollection
from protoplast.scrna.anndata.lightning_models import LinearClassifier
from protoplast.scrna.anndata.trainer import RayTrainRunner
from protoplast.scrna.anndata.torch_dataloader import DistributedAnnDataset
from protoplast.scrna.anndata.torch_dataloader import cell_line_metadata_cb, DistributedCellLineAnnDataset

from ray.train import Checkpoint
from ray.train.lightning import RayDDPStrategy

✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


root - INFO - Logging initialized. Current level is: INFO


In [2]:
from importlib.metadata import version

print(version("protoplast"))

0.1.2


## Training Set Overview
To start, we load the dataset used to train the example gene perturbation model featured in this notebook. This dataset will also serve as the basis for filtering non-targeting controls in the final prediction set, which we'll submit to the Virtual Cell Challenge.

In [3]:
file_paths = ["/mnt/hdd2/tan/competition_support_set_sorted/competition_train.h5"]
adata = anndata.read_h5ad(file_paths[0], backed = "r")
adata.obs.head(n = 5)

,target_gene,guide_id,batch,batch_var,cell_type
ACCTGGAAGATTATCCACTTTAGG-Flex_1_01,ACAT2,ACAT2_P1P2_A|ACAT2_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
AGCCTGTTCCTGGTTAACTTTAGG-Flex_1_01,ACAT2,ACAT2_P1P2_A|ACAT2_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
AGGCTTGAGTTTGACTACTTTAGG-Flex_1_01,ACAT2,ACAT2_P1P2_A|ACAT2_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
AGTCATAAGGTTAGACACTTTAGG-Flex_1_01,ACAT2,ACAT2_P1P2_A|ACAT2_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1
ATCCATCCATTTGGAGACTTTAGG-Flex_1_01,ACAT2,ACAT2_P1P2_A|ACAT2_P1P2_B,Flex_1_01,Flex_1_01,ARC_H1


## Define example perturbation model

In [4]:
import lightning.pytorch as pl
import torch
from torch import nn

class ExampleModel(pl.LightningModule):
    def __init__(self, num_genes, num_classes, control_pert, pert_names):
        super().__init__()
        self.control_pert = control_pert
        self.pert_names = pert_names
        self.embedding = nn.Embedding(num_embeddings = len(pert_names), embedding_dim = num_genes)
        self.output = nn.Linear(num_genes, num_classes)
    
    def forward(self, batch):
        embed = self.embedding(batch)
        out = self.output(embed).squeeze(1)
        return out

    def training_step(self, batch, batch_idx):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")        
        
        batch_pert_cell_counts = batch["pert_cell_counts"]
        X = batch_pert_cell_counts

        batch_pert_names = np.array(batch["pert_name"])
        sorted_idx = np.argsort(self.pert_names)
        pos = np.searchsorted(self.pert_names[sorted_idx], batch_pert_names)
        indices = torch.tensor(sorted_idx[pos]).unsqueeze(1).to(device)

        out = self(indices)
        mse_loss_fn = nn.MSELoss()
        loss = mse_loss_fn(out, X)
        
        self.log("train_loss", loss, on_step=True, prog_bar=True, sync_dist=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [5]:
class PerturbAnnDataset(DistributedAnnDataset):
    def transform(self, start: int, end: int):
        X = super().transform(start, end)

        # Metadata froms self.ad
        pert_names = self.ad.obs["target_gene"].iloc[start:end].astype(str).to_list()

        return {
            "pert_name": pert_names,
            "pert_cell_counts": X
        }

## Configure the model parameters

In [6]:
num_genes = adata.n_vars
output_size = adata.n_vars
control_pert = "non-targeting"

validation_genes = pd.read_csv("/mnt/hdd1/dung/tahoe100/pert_counts_Validation.csv").target_gene
pert_names = np.array(adata.obs.target_gene.unique().tolist() + validation_genes.tolist())

In [7]:
def metadata_cb(ad: anndata.AnnData, metadata: dict):
    metadata["num_genes"] = num_genes
    metadata["num_classes"] = output_size
    metadata["control_pert"] = control_pert
    metadata["pert_names"] = pert_names

## Create trainer & train perturbation model

In [8]:
trainer = RayTrainRunner(
    ExampleModel,
    PerturbAnnDataset,
    model_keys = ["num_genes",
                  "num_classes",
                  "control_pert",
                  "pert_names"],
    metadata_cb = metadata_cb,
    sparse_key = "X",
    runtime_env_config = {"working_dir": os.getcwd()},
)

2025-11-08 01:22:24,994	INFO worker.py:2003 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
2025-11-08 01:22:25,020	INFO packaging.py:588 -- Creating a file package for local module '/mnt/hdd1/dung/protoplast-ml-example/notebooks'.
2025-11-08 01:22:25,041	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_b1dc7d84a1987670.zip' (3.78MiB) to Ray cluster...
2025-11-08 01:22:25,063	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_b1dc7d84a1987670.zip'.
/mnt/hdd1/dung/protoplast-ml-example/.venv/lib/python3.11/site-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


In [9]:
thread_per_worker = 2
batch_size = 2000
test_size = 0.0
val_size = 0.0

result = trainer.train(
    file_paths = file_paths,
    batch_size = batch_size,
    test_size = test_size,
    val_size = val_size,
    thread_per_worker = thread_per_worker,  # 2
)

protoplast.scrna.anndata.trainer - INFO - Using 1 workers where each worker uses: {'CPU': 2, 'GPU': 1}
protoplast.scrna.anndata.strategy - INFO - Length of val_split: 0 length of test_split: 0, length of train_split: 6
protoplast.scrna.anndata.strategy - INFO - Length of after dropping remainder val_split: 0, length of test_split: 0, length of train_split: 6


(TrainController pid=3286748) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(TrainController pid=3286748) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


(TrainController pid=3286748) root - INFO - Logging initialized. Current level is: INFO
(TrainController pid=3286748) Attempting to start training worker group of size 1 with the following resources: [{'CPU': 2, 'GPU': 1}] * 1


(RayTrainWorker pid=3290138) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(RayTrainWorker pid=3290138) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


(RayTrainWorker pid=3290138) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=3290138) root - INFO - Logging initialized. Current level is: INFO
(TrainController pid=3286748) Started training worker group of size 1: 
(TrainController pid=3286748) - (ip=192.168.1.226, pid=3290138) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=3290138) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=3290138) GPU available: True (cuda), used: True
(RayTrainWorker pid=3290138) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=3290138) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=3290138) /mnt/hdd1/dung/protoplast-ml-example/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not 

...

(RayTrainWorker pid=3290138) 
(RayTrainWorker pid=3290138)   | Name      | Type      | Params | Mode 
(RayTrainWorker pid=3290138) ------------------------------------------------
(RayTrainWorker pid=3290138) 0 | embedding | Embedding | 3.6 M  | train
(RayTrainWorker pid=3290138) 1 | output    | Linear    | 326 M  | train
(RayTrainWorker pid=3290138) ------------------------------------------------
(RayTrainWorker pid=3290138) 330 M     Trainable params
(RayTrainWorker pid=3290138) 0         Non-trainable params
(RayTrainWorker pid=3290138) 330 M     Total params
(RayTrainWorker pid=3290138) 1,322.154 Total estimated model params size (MB)
(RayTrainWorker pid=3290138) 2         Modules in train mode
(RayTrainWorker pid=3290138) 0         Modules in eval mode
(RayTrainWorker pid=3290138) /mnt/hdd1/dung/protoplast-ml-example/.venv/lib/python3.11/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `devic

Epoch 0: 100%|██████████| 96/96 [00:36<00:00,  2.67it/s, v_num=0, train_loss=3.930]


(RayTrainWorker pid=3290138) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/dtran/protoplast_results/ray_train_run-2025-11-08_01-22-29/checkpoint_2025-11-08_01-23-43.156005)
(RayTrainWorker pid=3290138) Reporting training result 1: TrainingReport(checkpoint=Checkpoint(filesystem=local, path=/home/dtran/protoplast_results/ray_train_run-2025-11-08_01-22-29/checkpoint_2025-11-08_01-23-43.156005), metrics={'train_loss': 3.930330514907837, 'epoch': 0, 'step': 96}, validation_spec=None)


Epoch 0: 100%|██████████| 96/96 [01:04<00:00,  1.50it/s, v_num=0, train_loss=3.930]


(RayTrainWorker pid=3290138) `Trainer.fit` stopped: `max_epochs=1` reached.


In [10]:
ray.shutdown()

## Perturbation Prediction Using the Trained Model

With the model trained, we can now generate perturbation predictions. We'll start by loading the validation set, which provides the target genes and the number of predictions required for each gene.

In [11]:
validation_set = pd.read_csv("/mnt/hdd1/dung/tahoe100/pert_counts_Validation.csv")
validation_set.head(n = 5)

,target_gene,n_cells,median_umi_per_cell
0,SH3BP4,2925,54551.0
1,ZNF581,2502,53803.5
2,ANXA6,2496,55175.0
3,PACSIN3,2101,54088.0
4,MGST1,2096,54217.5


In [12]:
X = torch.tensor([]).long()
for row in validation_set.iloc:
    tmp = torch.tensor([np.where(pert_names == row.target_gene)][0] * row.n_cells)
    X = torch.cat((X, tmp), dim = 0)

/tmp/ipykernel_3279900/459944279.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  tmp = torch.tensor([np.where(pert_names == row.target_gene)][0] * row.n_cells)


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")        
X = X.to(device)

Load the trained model from the checkpoint path in previous `train()` step & make prediction

In [14]:
model = ExampleModel.load_from_checkpoint(result.checkpoint.path + "/checkpoint.ckpt", 
                                          num_genes = num_genes, 
                                          num_classes = output_size, 
                                          control_pert = control_pert, 
                                          pert_names = pert_names)
model.eval()  # set to eval mode

ExampleModel(
  (embedding): Embedding(201, 18080)
  (output): Linear(in_features=18080, out_features=18080, bias=True)
)

In [15]:
pred = model(X)
print(pred)

tensor([[-0.1231,  0.1643, -1.6772,  ...,  0.8463,  1.7868,  0.9581],
        [-0.1231,  0.1643, -1.6772,  ...,  0.8463,  1.7868,  0.9581],
        [-0.1231,  0.1643, -1.6772,  ...,  0.8463,  1.7868,  0.9581],
        ...,
        [-0.9791,  0.2159, -0.8821,  ..., -0.0454,  1.3044,  0.7011],
        [-0.9791,  0.2159, -0.8821,  ..., -0.0454,  1.3044,  0.7011],
        [-0.9791,  0.2159, -0.8821,  ..., -0.0454,  1.3044,  0.7011]],
       device='cuda:0', grad_fn=<SqueezeBackward1>)


## Creating the Predicted AnnData
Next, we generate an `AnnData` object from the model’s predicted gene perturbations. To complete the dataset, we also include the non-targeting control group. Since these controls aren't part of the prediction, we'll copy them directly from the training set.

In [16]:
sample_predicted_adata = anndata.AnnData(
    X = pred.cpu().detach().numpy(),
    obs = pd.DataFrame(
        {
            "target_gene": np.repeat(validation_set.target_gene, validation_set.n_cells).tolist(),
        },
        index = np.arange(validation_set.n_cells.sum()).astype(str),
    ),
    var = pd.DataFrame(index = list(adata.var_names)),
)

In [17]:
sample_submission = anndata.concat([adata[adata.obs["target_gene"] == "non-targeting"], 
                                    sample_predicted_adata])


In [18]:
sample_submission.write_h5ad("/mnt/hdd1/dung/tahoe100/prediction.h5ad")

... storing 'target_gene' as categorical


## Running `cell-eval`

We’ll now use `cell-eval` to process the `AnnData` object and prepare it for submission to the competition.

In [19]:
!cell-eval prep -i /mnt/hdd1/dung/tahoe100/prediction.h5ad -g /mnt/hdd1/dung/tahoe100/gene_names.csv

INFO:cell_eval._cli._prep:Reading input anndata
INFO:cell_eval._cli._prep:Reading gene list
INFO:cell_eval._cli._prep:Preparing anndata
INFO:cell_eval._cli._prep:Using 32-bit float encoding
INFO:cell_eval._cli._prep:Setting data to sparse if not already
INFO:cell_eval._cli._prep:Simplifying obs dataframe
INFO:cell_eval._cli._prep:Simplifying var dataframe
INFO:cell_eval._cli._prep:Creating final minimal AnnData object
INFO:cell_eval._cli._prep:Applying normlog transformation if required
INFO:cell_eval._evaluator:Input is found to be log-normalized already - skipping transformation.
INFO:cell_eval._cli._prep:Writing h5ad output to /tmp/tmp65q0mox8/pred.h5ad
INFO:cell_eval._cli._prep:Zstd compressing /tmp/tmp65q0mox8/pred.h5ad
/tmp/tmp65q0mox8/pred.h5ad :  4.58%   (  10.7 GiB =>    501 MiB, /tmp/tmp65q0mox8/pred.h5ad.zst) 
INFO:cell_eval._cli._prep:Packing files into /mnt/hdd1/dung/tahoe100/prediction.prep.vcc
INFO:cell_eval._cli._prep:Done
